# 2.- Creación del dataset

<a target="_blank" href="https://colab.research.google.com/github/Chiriviki/congreso/blob/8772b4a6f99879ca8a1f0538322470367bf4618b/2.-%20Crea_dataset.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

Este cuaderno toma los textos extraídos y trata de:
- Extraer metadatos y anexarlos a cada intervención.
- Limpiar los textos.
- Cruzar los nombres de políticos con el partido político al que pertenecen.

## Carga corpus

En primer lugar carga el corpus.

In [ ]:
import os
import json

carpeta = 'dataset/corpus_v1'
 
corpus = []
for filename in os.listdir(carpeta):
    f = os.path.join(carpeta, filename)
    
    if os.path.isfile(f):
        with open(f, "r") as f_r:
            f_json = json.load(f_r)
            corpus.append(f_json)

            

In [ ]:
corpus[0]

{'datos': 'DS. Congreso de los Diputados, Pleno y Dip. Perm., núm. 1, de 03/12/2019',
 'intervenciones': [{'name1': 'PRESIDENTE DE LA MESA DE EDAD',
   'name2': 'Zamarrón Moreno',
   'span': [5107, 5165],
   'texto': ' Señorías, se abre la sesión.\nEn virtud de lo dispuesto en el artículo 2 del Reglamento del Congreso de los Diputados, la Mesa de Edad ha quedado constituida por el diputado electo de mayor edad de los presentes, don Agustín Zamarrón\nMoreno -es decir, yo mismo-, como presidente, y por las dos más jóvenes como secretarias, a saber, doña Marta Rosique i Saltor y doña Lucía Muñoz Dalda, según los datos que constan en la Cámara.\nSeñorías, el artículo 99 de nuestra Constitución expone el artificioso modo para el nombramiento de presidente de Gobierno, dando inicio a un proceso que culmina en un Gobierno legítimo y pleno en sus atribuciones; al hacerlo determina la\ngrave responsabilidad de los intervinientes en el proceso, en lo que afecta a la responsabilidad de las señora

## Extraer metadatos 

Los metadatos se encuentran a en una cadena de caracetres. La siguiente función extrae los distintos metadatos de esta cadena.


In [ ]:
from datetime import datetime

# Extraer los datos de la sesión

def extract_data(data_str):
    
    # Separa los elementos por coma
    data_splitted = data_str.split(",")
    
    # Los dos primeros campo los cogemos tal cual
    data ={ "cámara": data_splitted[0],
          "organismo": data_splitted[1]}
    
    # Extrae el número de sesión
    num = int(data_splitted[2][5:])    
    data["numero"] = num
    
    # Extrae fecha - se queda como str para dejarlo a gusto del consumidor
    fecha = data_splitted[3][4:]
    data["fecha"] = fecha
    return data

extract_data(corpus[0]["datos"])

{'cámara': 'DS. Congreso de los Diputados',
 'organismo': ' Pleno y Dip. Perm.',
 'numero': 1,
 'fecha': '03/12/2019'}

Con estos datos ya podemos crear una estrutura tabular.

In [ ]:
import pandas as pd
dataset_df = pd.DataFrame([extract_data(sesion["datos"]) | {"orden_interv":i, "name1": interv["name1"], "name2": interv["name2"], "texto":interv["texto"]} for sesion in corpus for i, interv in enumerate(sesion["intervenciones"])])
        

In [ ]:
dataset_df.head()

cámara            organismo  numero       fecha  \
0  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
1  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
2  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
3  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
4  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   

   orden_interv                          name1             name2  \
0             0  PRESIDENTE DE LA MESA DE EDAD   Zamarrón Moreno   
1             1  PRESIDENTE DE LA MESA DE EDAD   Zamarrón Moreno   
2             2  SECRETARIA DE LA MESA DE EDAD  Rosique i Saltor   
3             3  PRESIDENTE DE LA MESA DE EDAD   Zamarrón Moreno   
4             4  SECRETARIA DE LA MESA DE EDAD  Rosique i Saltor   

                                               texto  
0   Señorías, se abre la sesión.\nEn virtud de lo...  
1   De conformidad con lo dispuesto en el artícul...  
2   Artículo 5 del Real Decreto 551/2019, de 24 d...  
3   Las señoras secretarias de la Mesa procederán...  
4   Junqueras i Vies, Oriol; Forn Chiariello, Joa...

## Limpieza
Lo único que necesaitamos hacer es reemplazar los saltos de líneas por espacios.

In [ ]:
dataset_df["texto"] = dataset_df["texto"].str.replace("\n", " ")
dataset_df.head()

cámara            organismo  numero       fecha  \
0  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
1  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
2  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
3  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   
4  DS. Congreso de los Diputados   Pleno y Dip. Perm.       1  03/12/2019   

   orden_interv                          name1             name2  \
0             0  PRESIDENTE DE LA MESA DE EDAD   Zamarrón Moreno   
1             1  PRESIDENTE DE LA MESA DE EDAD   Zamarrón Moreno   
2             2  SECRETARIA DE LA MESA DE EDAD  Rosique i Saltor   
3             3  PRESIDENTE DE LA MESA DE EDAD   Zamarrón Moreno   
4             4  SECRETARIA DE LA MESA DE EDAD  Rosique i Saltor   

                                               texto  
0   Señorías, se abre la sesión. En virtud de lo ...  
1   De conformidad con lo dispuesto en el artícul...  
2   Artículo 5 del Real Decreto 551/2019, de 24 d...  
3   Las señoras secretarias de la Mesa procederán...  
4   Junqueras i Vies, Oriol; Forn Chiariello, Joa...

In [ ]:
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46062 entries, 0 to 46061
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   cámara        46062 non-null  object
 1   organismo     46062 non-null  object
 2   numero        46062 non-null  int64 
 3   fecha         46062 non-null  object
 4   orden_interv  46062 non-null  int64 
 5   name1         46062 non-null  object
 6   name2         11123 non-null  object
 7   texto         46062 non-null  object
dtypes: int64(2), object(6)
memory usage: 2.8+ MB


## Anexa partido

Los datos de los partidos se encuentran en dos archivos en formato de tabla extraídos de wikipedia.

A continuación se meustran las características que dispone. En nuestro caso solo necesitamos el nombre y el grupo político al que pertenece.

In [ ]:
diputados = pd.read_csv("./dataset/Anexo_Diputados_de_la_XIV_legislatura_de_EspaB1a_2.csv")
diputados_baja = pd.read_csv("./dataset/Anexo_Diputados_de_la_XIV_legislatura_de_EspaB1a_causaronbaja.csv")

In [ ]:
diputados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355 entries, 0 to 354
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Nombre y apellidos  355 non-null    object 
 1   Grupo               0 non-null      float64
 2   Grupo.1             355 non-null    object 
 3   Lista               0 non-null      float64
 4   Lista.1             355 non-null    object 
 5   Circunscripción     355 non-null    object 
 6   Alta                355 non-null    object 
dtypes: float64(2), object(5)
memory usage: 19.5+ KB


In [ ]:
diputados_baja.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Nombre y apellidos  41 non-null     object 
 1   Grupo               0 non-null      float64
 2   Grupo.1             41 non-null     object 
 3   Lista               0 non-null      float64
 4   Lista.1             41 non-null     object 
 5   Circunscripción     41 non-null     object 
 6   Alta                41 non-null     object 
 7   Baja                41 non-null     object 
 8   Sustituto/a         41 non-null     object 
dtypes: float64(2), object(7)
memory usage: 3.0+ KB


Une ambos conjuntos de datos.

In [ ]:
diputados_df = pd.concat([diputados, diputados_baja])[["Nombre y apellidos", "Grupo.1", "Lista.1", "Circunscripción", "Alta", "Baja"]]

El nombre aparece en formato "nombre, apellido1 apellido2". Para curzar los datos necesitamos únicamente los apellidos.

In [ ]:
diputados_df[["Apellidos", "Nombre"]] = diputados_df["Nombre y apellidos"].str.split(", ", expand=True)
diputados_df.drop(columns="Nombre y apellidos", inplace=True)

In [ ]:
diputados_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396 entries, 0 to 40
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Grupo.1          396 non-null    object
 1   Lista.1          396 non-null    object
 2   Circunscripción  396 non-null    object
 3   Alta             396 non-null    object
 4   Baja             41 non-null     object
 5   Apellidos        396 non-null    object
 6   Nombre           396 non-null    object
dtypes: object(7)
memory usage: 24.8+ KB


### Unión

Para unirlos se realiza las siguientes tareas:
- Se transofrma todo a mayúsculas.
- Se utiliza name2 si no es nulo, en tal caso se utiliza name1.
- Se curzan mediante Leftjoin

In [ ]:
# Si name2 es nulo, lo sustituimos por name1

dataset_df['name2'] = dataset_df['name2'].fillna(dataset_df['name1'])

# Mayus
dataset_df["name2"] = dataset_df["name2"].str.upper()

diputados_df["Apellidos"] = diputados_df["Apellidos"].str.upper()

In [ ]:
merged_partidos_name2 = dataset_df.merge(diputados_df, how="left", left_on="name2", right_on="Apellidos")

In [ ]:
merged_partidos_name2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46859 entries, 0 to 46858
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   cámara           46859 non-null  object
 1   organismo        46859 non-null  object
 2   numero           46859 non-null  int64 
 3   fecha            46859 non-null  object
 4   orden_interv     46859 non-null  int64 
 5   name1            46859 non-null  object
 6   name2            46859 non-null  object
 7   texto            46859 non-null  object
 8   Grupo.1          26519 non-null  object
 9   Lista.1          26519 non-null  object
 10  Circunscripción  26519 non-null  object
 11  Alta             26519 non-null  object
 12  Baja             1400 non-null   object
 13  Apellidos        26519 non-null  object
 14  Nombre           26519 non-null  object
dtypes: int64(2), object(13)
memory usage: 5.7+ MB


De las 48000 intervenciones, solo ha podido cruzar unas 26000. Pese a los erroes, el número de itnervenciones es suficiente. La mayoría de los cruces erróneos son de la presidenta del congreso.

In [ ]:
errores = merged_partidos_name2[merged_partidos_name2["Grupo.1"].isna()]

In [ ]:
errores["name2"].value_counts()

PRESIDENTA              18687
CALVIÑO SANTAMARÍA        308
BOLAÑOS GARCÍA            140
DARIAS SAN SEBASTIÁN      105
ESCRIVÁ BELMONTE           95
                        ...  
LEGARDE URIARTE             1
QUEVEDO IBURBE              1
GRANDE MARLASKA             1
ROMANÍ CANTERA              1
ÁNGULO ROMERO               1
Name: name2, Length: 229, dtype: int64

## Guardar

In [ ]:
merged_partidos_name2.to_csv("dataset/dataset_v1_1/dataset_v1_1.csv")